# Modèle de langue et word embeddings

Les modèles de langue sont des modèles probabilistes d'une langue. A partir d'un corpus d'étude, ils apprennent la probabilité d'apparition d'un terme, ainsi que la probabilité qu'un terme suive un autre terme ou séquence de termes. Ainsi, les modèles de langues appartiennent à la famille des méthodes non-supervisées. Ils sont particulièrement adapté pour étudier la structure d'une langue, mais aussi adaptés à des tâches comme la reconnaissance de langue, l'analyse en partie du discours ou la génération de texte. 

Les premiers modèles de language reposent sur des méthodes statistiques. Plus récémment, les principaux modèles de langues tels que BERT ou GPT reposent sur les méthodes neuronales. Si ces derniers sont bien plus puissants et efficaces, les ressources informatiques nécessaires pour les faire tourner sont bien plus importantes.  

Dans ce cours, nous allons entraîner un modèle de langue statistique à l'aide de la librairie NLTK. 

In [1]:
!pip install nltk

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
     |████████████████████████████████| 294 kB 6.6 MB/s eta 0:00:01


## Modèle n-gramme

Un modèle n-gramme détermine le mot suivant le plus probable à partir d'une séquence n-1 de mots. Par exemple, un modèle bigramme déterminera la probabilité qu'un mot apparaisse en fonction du mot précédent, un trigramme à partir des deux mots précédents, et ainsi de suite. Un modèle unigramme lui ne repose que sur la probabilité d'apparition du mot dans le corpus, sans prendre en compte ce qui précéde. 

Exemple : 

p(There was heavy rainfall) = p(START, There, was, heavy, rainfall, END) = p(There|START)p(was|There)p(heavy|There was)p(rainfall|There was heavy)p(END|There was heavy rainfall)

Cependant, on ne peut réellement calculer la probabilité d'apparition d'une séquence aussi longue. On peut cependant calculer les probabilités d'apparition de chaque n-gramme précédents. Ainsi : 

p(There was heavy rainfall) = p(There|START)p(was|There)p(heavy|was)p(rainfall|heavy)p(END|rainfall)

Pour la génération de texte, les modèles à partir de trigrammes sont les plus intéressants. Cependant, la taille du modèle dépend de la tâche et du type de données traité.

### Préparer les données

Pour entraîner un modèle, il nous faut des liste de documents tokénisé. Ci-dessous, la variable "corpus" contient deux "phrases"

In [1]:
corpus = [
    ['a', 'b', 'c'],
    ['a', 'c', 'd', 'c', 'e', 'f']
    ]

NLTK permet de rapidement constituer des n-grammes. Ci-dessous, on produit des bigrammes à partir de nos séquences

In [4]:
from nltk.util import bigrams, trigrams

print('Bigrams')
for x in corpus:
    gram = list(bigrams(x))
    print(gram)

print('Trigrams')
for x in corpus:
    gram = list(trigrams(x))
    print(gram)

Bigrams
[('a', 'b'), ('b', 'c')]
[('a', 'c'), ('c', 'd'), ('d', 'c'), ('c', 'e'), ('e', 'f')]
Trigrams
[('a', 'b', 'c')]
[('a', 'c', 'd'), ('c', 'd', 'c'), ('d', 'c', 'e'), ('c', 'e', 'f')]


Il est important de distinguer l'occurrence d'un mot au sein de la phrase d'au début ou de la fin de phrase. Pour cela, on ajoute à la séquence deux caractères spéciaux, **s** et **/s** pour indiquer respectivement le début et la fin de phrase. 

In [5]:
from nltk.util import pad_sequence

for x in corpus:
    # l'argument n précise que l'on travaille sur des bigrammes
    seq = pad_sequence(x, n=2, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>')
    print(list(seq))

['<s>', 'a', 'b', 'c', '</s>']
['<s>', 'a', 'c', 'd', 'c', 'e', 'f', '</s>']


La fonction "pad_both_ends" facilite l'emploi de "pad_sequence". Ci-dessous, une fonction pour prétraiter un corpus. Notez l'argument "gram_func" qui attend une fonction pour transformer la séquence en n-gramme (ex: fonction bigrams ou trigrams de NLTK). L'argument "n" doit correspondre au n-gramme donné.

In [6]:
from nltk.lm.preprocessing import pad_both_ends

for x in corpus:
    # l'argument n précise que l'on travaille sur des bigrammes
    seq = pad_both_ends(x, n=2)
    print(list(seq))

['<s>', 'a', 'b', 'c', '</s>']
['<s>', 'a', 'c', 'd', 'c', 'e', 'f', '</s>']


Afin d'avoir un modèle plus robuste, on peut l'entraîner sur des bigrammes et unigrammes à la fois. NLTK met à disposition la fonction "everygram", qui génère tous les n-grammes d'une séquence jusqu'à n.

In [7]:
from nltk.util import everygrams
n = 3
for x in corpus:
    # l'argument n précise que l'on travaille sur des bigrammes
    padded_seq = pad_both_ends(x, n=n)
    padded_grams = list(everygrams(padded_seq, max_len=n))
    print(padded_grams)

[('<s>',), ('<s>', '<s>'), ('<s>', '<s>', 'a'), ('<s>',), ('<s>', 'a'), ('<s>', 'a', 'b'), ('a',), ('a', 'b'), ('a', 'b', 'c'), ('b',), ('b', 'c'), ('b', 'c', '</s>'), ('c',), ('c', '</s>'), ('c', '</s>', '</s>'), ('</s>',), ('</s>', '</s>'), ('</s>',)]
[('<s>',), ('<s>', '<s>'), ('<s>', '<s>', 'a'), ('<s>',), ('<s>', 'a'), ('<s>', 'a', 'c'), ('a',), ('a', 'c'), ('a', 'c', 'd'), ('c',), ('c', 'd'), ('c', 'd', 'c'), ('d',), ('d', 'c'), ('d', 'c', 'e'), ('c',), ('c', 'e'), ('c', 'e', 'f'), ('e',), ('e', 'f'), ('e', 'f', '</s>'), ('f',), ('f', '</s>'), ('f', '</s>', '</s>'), ('</s>',), ('</s>', '</s>'), ('</s>',)]


Le modèle nécessite également un vocabulaire, c'est à dire d'un ensemble de mots connus du modèle. Les symboles de début et fin de phrases font partie de ce vocabulaire.

In [17]:
vocab = [token for sent in text for token in pad_both_ends(sent, n=2)]
vocab

['<s>', 'a', 'b', 'c', '</s>', '<s>', 'a', 'c', 'd', 'c', 'e', 'f', '</s>']

La fonction "padded_everygram_pipeline" nous permet de procéder à toutes ces étapes à l'aide d'une seule fonction

In [14]:
from nltk.lm.preprocessing import padded_everygram_pipeline
n = 2
data, vocab = padded_everygram_pipeline(n, corpus)

In [12]:
for x in data:
    for y in x:
        print(y)

('<s>',)
('<s>', 'a')
('a',)
('a', 'b')
('b',)
('b', 'c')
('c',)
('c', '</s>')
('</s>',)
('<s>',)
('<s>', 'a')
('a',)
('a', 'c')
('c',)
('c', 'd')
('d',)
('d', 'c')
('c',)
('c', 'e')
('e',)
('e', 'f')
('f',)
('f', '</s>')
('</s>',)


In [13]:
for x in vocab:
    print(x)

<s>
a
b
c
</s>
<s>
a
c
d
c
e
f
</s>


## Entraîner le modèle

Nous pouvons désormais entraîner un modèle de langue. Nous allons entraîner un modèle MLE (Maximum Likelihood Estimator), qui compte la fréquence de chaque n-gramme puis les normalise pour que ces valeurs soient contenues entre 0 et 1. Pour plus de détails sur le fonction, voir Jurafsky. 

In [15]:
from nltk.lm import MLE
n = 2
lm = MLE(n)
# le vocabulaire est vide au départ
len(lm.vocab)

0

In [16]:
lm.fit(data, vocab)
len(lm.vocab)

9

In [17]:
text

NameError: name 'text' is not defined

In [18]:
print(lm.vocab.lookup(corpus[0]))
# les mots "aliens", "from", "Mars" ne sont pas contenus dans le vocabulaire. Ils sont donc associés à un token UNK
print(lm.vocab.lookup(["aliens", "from", "Mars"]))

('a', 'b', 'c')
('<UNK>', '<UNK>', '<UNK>')


In [19]:
# "a" apparait 2 fois dans le corpus
print('Compte de "a" ', lm.counts['a'])

# la probabilité que "a" apparaîsse dans le corpus
print(lm.score("a"))

# la probabilité que "b" apparaîsse en étant précédé de "a" == 1/2
print(lm.score("b", ["a"]))

# la probabilité que "d" apparaîsse en étant précédé de "a" == aucune
print(lm.score("d", ["a"]))

Compte de "a"  2
0.15384615384615385
0.5
0.0


## Evaluer un modèle de langue

Comme pour l'apprentissage supervisé, il nous faut des données de test pour évaluer un modèle. Cependant, le modèle de langue ne peut s'évaluer en terme de Précision, Rappel et F1, puisqu'il ne s'agit ni de classer, ni de retourner des informations. 

Pour évaluer un modèle de langue, on se sert de la mesure de **perplexité**, c'est à dire la probabilité ou la surprise que les données de test correspondent au modèle de langue. Elle repose sur la notion d'entropie, c'est-à-dire d'incertitude, dans la théorie de l'information. Plus la perplexité est basse (donc la probabilité est haute), et plus les données de test correspondent. Ainsi, pour évaluer un modèle de langue, il nous faut des extraits de langues qui sont représentatifs. 

La perplexité n'est pas la seule mesure pour évaluer un modèle de langue, mais s'en est une des principales. 

In [20]:
# ces données correspondent à notre modèle
test = [('a', 'b'), ('c', 'd')]
print(lm.perplexity(test))

# ces données ne correspondent pas à notre modèle: le modèle est inf(iniment) surpris
test = [('a', 'c'), ('b', 'd')]
print(lm.perplexity(test))

test = [('1', '2'), ('3', '4')]
print(lm.perplexity(test))

2.449489742783178
inf
inf


## Générer un texte

La force majeure des modèles de langues est de pouvoir générer des séquences de tokens (mots, POS tags...). On peut laisser le modèle tout générer seul ou bien lui donner une séquence de départ. De plus, on peut intégrer des règles pour modifier le processus de génération de texte. 

In [21]:
# génération simple
print(lm.generate(5, random_seed=42))

# génération en spécifiant le début de la séquence
print(lm.generate(5, text_seed=['<s>'],random_seed=42))

print(lm.generate(5, text_seed=['<s>', 'a'],random_seed=42))

['c', '</s>', '<s>', 'a', 'c']
['a', 'b', 'c', '</s>', 'c']
['c', '</s>', '<s>', 'a', 'c']


## Smoothing

Pour le moment, notre modèle n'est capable d'assigner une probabilité à une séquence seulement s'il l'a vu dans le corpus d'étude. Cependant, un caractère peut être présent dans le vocabulaire, mais apparaître dans un nouveau contexte dans un autre corpus. Un caractère peut également ne pas apparaître du tout dans le vocabulaire du modèle. Dans un tel cas, ce caractère aura une probabilité nulle qui lui sera assignée. Pour éviter cela, il faut procéder au **smoothing**. Les principaux algorithmes de smoothing sont :

* Laplace (add-one) smoothing
* add-k smoothing
* stupid backoff
* Kneser-Ney smoothing

Une autre possibilité est d'employer l'**interpolation** ou le **backoff**

* backoff : si une séquence n'existe pas, on réduit cette séquence jusqu'à en trouver une connue du modèle
* interpolation : on fait la somme de la probabilité associée à chaque n-gramme contenu dans la séquence (unigrame + bigramme + ... + n-gramme)

In [22]:
from nltk.lm import Laplace
n = 2

data, vocab = padded_everygram_pipeline(n, corpus)

lm = Laplace(n)
lm.fit(data, vocab)

In [23]:
# ces données correspondent à notre modèle
test = [('a', 'b'), ('c', 'd')]
print(lm.perplexity(test))

# ces données ne correspondent pas à notre modèle: le modèle est inf(iniment) surpris
test = [('a', 'c'), ('b', 'd')]
print(lm.perplexity(test))

test = [('1', '2'), ('3', '4')]
print(lm.perplexity(test))

5.744562646538029
7.416198487095664
9.000000000000002


In [24]:
from nltk.lm import AbsoluteDiscountingInterpolated
n = 2

data, vocab = padded_everygram_pipeline(n, corpus)

lm = AbsoluteDiscountingInterpolated(order=n)
lm.fit(data, vocab)

In [25]:
# ces données correspondent à notre modèle
test = [('a', 'b'), ('c', 'd')]
print(lm.perplexity(test))

# ces données ne correspondent pas à notre modèle: le modèle est inf(iniment) surpris
test = [('a', 'c'), ('b', 'd')]
print(lm.perplexity(test))

test = [('1', '2'), ('3', '4')]
print(lm.perplexity(test))

6.2300398978808
7.62564998111037
inf


## Ressources : 

Pour plus de détails dans les calculs de probabilités et la construction mathématique des modèles de langue : 
* Documentation NLTK : https://www.nltk.org/api/nltk.lm.html
* Jurafsky et al : https://web.stanford.edu/~jurafsky/slp3/3.pdf